# GPT-4 for Diseases Understanding

In [ ]:
import json
from costs import GPT4
from api_gpt import LargeLanguageModel, PARAMS, eval_pipline
from modelss import Message, Sample, SetEvalDocs, Promt
import sys
sys.path.append("../../")
from utils import get_tui_columns, load_samples_csv, load_promts_GPT

## Set-up Experiment
---

In [ ]:
BASE_PATH = "../../"
PATH_GOLDS = BASE_PATH + "standard/golds_v4.json"
PROMT_PATH = BASE_PATH + "promts/"
PATH_PROMPTS = PROMT_PATH + "prompts.xlsx"
PATH_EXTERNAL_INFO = PROMT_PATH + "tuis_desc.csv"
PATH_SAMPLES = PROMT_PATH + "samples.csv"

### Load Golds

In [ ]:
golds = []

with open(PATH_GOLDS) as f:
    golds = json.load(f)

print(len(golds))

### Load External Information

In [ ]:
tuis = get_tui_columns(PATH_EXTERNAL_INFO, columns='name')
print(tuis)

### Load Prompts

In [ ]:
prompts = load_promts_GPT(PATH_PROMPTS)
prompts

In [ ]:
prompt = prompts[4]
behave = prompt.msg
behave_e = behave.copy()
behave['content'] = behave['content'].replace('{tuis}', tuis)
behave

In [ ]:
texts, golds_samples = load_samples_csv(PATH_SAMPLES)
samples = []
for text, symptoms in zip(texts, golds_samples):
    user_message = Message(role="user", content=text)
    assistant_message = Message(role="assistant", content=symptoms)
    sample = Sample(user=user_message, agent=assistant_message)
    samples.append(sample)

In [ ]:
samples

### Load Model


In [ ]:
llms = [LargeLanguageModel(model=GPT4, **PARAMS)]

## Create Eval Sets

In [ ]:
sets_eval = [SetEvalDocs({"name": "results_diseases_gpt4" , "golds": golds})]

In [ ]:
promts2test = [Promt(behave=behave, name="Zero Shot"), Promt(behave=behave, name="1 Few-shot", samples=samples[:1]), Promt(behave=behave, name="3 Few-shot", samples=samples[:3]), Promt(behave=behave, name="5 Few-shot", samples=samples[:5]), Promt(behave=behave, name="10 Few-shot", samples=samples[:10])]
promts2test

In [ ]:
results = eval_pipline(llms, sets_eval, promts2test)

In [ ]:
results

# External Knowledge

In [ ]:
tuis = get_tui_columns(PATH_EXTERNAL_INFO, columns='both')
print(tuis)

In [ ]:
behave_e['content'] = behave_e['content'].replace('{tuis}', tuis)
behave_e

In [ ]:
sets_eval_external = [SetEvalDocs({"name": "results_diseases_gpt4" , "golds": golds})]

In [ ]:
promts2test_e = [Promt(behave=behave_e, name="Zero Shot"), Promt(behave=behave_e, name="1 Few-shot", samples=samples[:1]), Promt(behave=behave_e, name="3 Few-shot", samples=samples[:3]), Promt(behave=behave_e, name="5 Few-shot", samples=samples[:5]), Promt(behave=behave_e, name="10 Few-shot", samples=samples[:10])]
promts2test_e

In [ ]:
results_e = eval_pipline(llms, sets_eval_external, promts2test_e)

In [ ]:
results_e

In [ ]:
from utils import dump_results
path = "../../results/gpt4_external_results" + ".json"
dump_results(results_e, path)